 [必須事項 #251: 出来高が急に増えて価格が上がった株を素直に買ってみる - After large price change - Redmine](http://116.58.169.125:6101/issues/251)
 + [参照](https://vantagepointtrading.com/wp-content/uploads/2010/05/How-I-Made-2000000-in-the-Stock-Market.pdf)
 + いわゆる， Darvas Box 
 + 急にVolumeが増えて価格が上昇した株をトレンドフォローするストラテジー
 + 定義１：どのくらいのボリュームからどのくらい上がったか？
 + 定義２：ホールド期間
 + 定義３：ポジションオープンのタイミング
 
 ![scale](https://www.evernote.com/shard/s6/sh/a6ca67c6-84e0-4559-8671-70b5287fb14b/d1b7dd0d631a9c30ea620bcbc4958047/res/da1a1b01-fd11-4135-adbf-7cbcb8ae87f8/How-I-Made-2000000-in-the-Stock-Market.pdf.jpg?resizeSmall&width=832
)
+ （A)のタイミングを確認したあと，（B)でホールドして，（C)でロスカットするも，その後の動きで，間違いないと思い，（D)でもう一度ロングしたそうです．
+ VWAP１００くらいで定義１を検証してみる．
 

# [Bulkowski's Darvas Box](http://thepatternsite.com/Darvas.html)

+ ETFで，WeeklyScaleでトレードするとワークする（勝率49％で，年率10.5％）　
+ DDでかい
+ ストラテジー
   
    1. 過去一年のHighを見つけ，その後3日間（Highが出た日を含めたら4日間）がそのHighを超えていない銘柄を見つける
    1. その期間のLowを確認
    1. その後，終値がそのBoxの上限を突き抜けたら，次の日のOpenで買い，下限を抜けたら次の日のOpenで売る
    1. 日中にBoxの下限上限を越えたが，CloseはBox内に収まった場合，新しくBoxを作成し直す
        
    
    

#  [Darvas Box (DBOX) | Linn Software](https://www.linnsoft.com/techind/darvas-box-dbox)

 ![msft](https://www.linnsoft.com/sites/default/files/techind/darvaschart.gif)

In [ ]:
from quantopian.pipeline import Pipeline,CustomFactor
from quantopian.research import run_pipeline
from zipline.api import attach_pipeline, pipeline_output  
from quantopian.pipeline.factors import AverageDollarVolume,Returns

from quantopian.pipeline.data.builtin import USEquityPricing

from quantopian.pipeline.filters import Q1500US, Q500US

import numpy as np
import pandas as pd
import matplotlib.cm as cm


In [ ]:
def initialize(context):
    pype = make_pipeline()
    attach_pipeline(pype, name='mypype')
    

class RollingMax(CustomFactor):
    inputs = [USEquityPricing.high]
    def compute(self, today, asset_ids, out, values): 
        out[:] = np.nanmax(values,axis=0) 

class RollingMin(CustomFactor):
    inputs = [USEquityPricing.low]
    def compute(self, today, asset_ids, out, values): 
        out[:] = np.nanmin(values,axis=0) 

class Continuously_true(CustomFactor):
    window_safe = True
    
    def compute(self, today, asset_ids, out, values, ): 
        out[:] = np.all(values, axis=0) #np.all(values, axis=0)
        
        
def make_pipeline():
    base_universe = Q1500US()
    close = USEquityPricing.close
    volume = USEquityPricing.volume
    rollingmax10 = RollingMax(window_length=10)
    rollingmin10 = RollingMin(window_length=10)
    dollar_volume = AverageDollarVolume(window_length=250)
    yesterday_dollar_volume = AverageDollarVolume(window_length=1)
    bikkuri_volume_change = yesterday_dollar_volume / dollar_volume - 1 
    high_bikkuri_volume_change = bikkuri_volume_change.percentile_between(90, 100)
    is_max = rollingmax10.eq(close)
    ret = Returns(window_length=2)
    
    
    #threedays_max = Continuously_true(inputs=[is_max], window_length=3)
    
    pipe = Pipeline(columns={'close': close.latest, 
                             'high': USEquityPricing.high.latest,
                            'rollingmax10': rollingmax10,
                            'rollingmin10': rollingmin10,
                             'dollar_volume': dollar_volume,
                             'yesterday_dollar_volume':yesterday_dollar_volume,
                             'bikkuri_volume_change': bikkuri_volume_change, 
                             'return':ret,
                            #'is_max': is_max,
                             #'threedays_max': threedays_max,
                            },
                   screen=base_universe & high_bikkuri_volume_change )
    return pipe 


results = run_pipeline(make_pipeline(), start_date='2015-1-1', end_date='2015-3-1')

    

In [ ]:
dates = results.index.get_level_values(0).unique()
print dates[0]
results.ix[dates[0]]

In [ ]:
## sp500ish の銘柄を取得するだけのためにパイプライン

def make_pipeline():
    base_universe = Q500US()
    close = USEquityPricing.close
    pipe = Pipeline(
        columns={'close': close.latest, },
        screen=base_universe)
    return pipe 

results = run_pipeline(make_pipeline(), start_date='2017-7-5', end_date='2017-7-6')

stocks = results.ix['2017-07-05'].index
print (len(stocks))
data = get_pricing(stocks, start_date='2013-01-03', end_date='2017-01-03', 
                   frequency='daily', fields='price', handle_missing='raise')



In [ ]:
data

In [ ]:
df_pct_change = data.pct_change().ix[1:]
df_2_5_change = data.ix[1:].pct_change(40)
logreturn = (data / data.shift(1)).apply(np.log)
df_vol = logreturn.rolling(window=5,center=False).std().ix[1:]

In [ ]:
import matplotlib.pyplot as plt 
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.scatter(df_pct_change[symbols('HP')], 
               df_2_5_change[symbols('HP')],
           c=df_vol[symbols('HP')],
          
          cmap=cm.Reds)


In [ ]:
fig = plt.figure(figsize=(10,10))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)

data1 = data.ix[:, :10]


df_pct_change = data1.pct_change().ix[1:]
df_2_5_change = data1.ix[1:].pct_change(5)

logreturn = (data1 / data1.shift(1)).apply(np.log)
df_vol = logreturn.rolling(window=5,center=False).std().ix[1:]

num = len(data1.columns)
xs = 5
ys = int(num/5)+1

for i, c in enumerate(data1.columns):
    x = i % xs
    y = int(i/xs)+ys
    ax = fig.add_subplot(xs, ys, i+1)
    ax.set_title(c.symbol)
    ax.scatter(df_pct_change[symbols(c)], 
               df_2_5_change[symbols(c)],
               c=df_vol[symbols(c)],cmap=cm.Reds
               )
    

In [ ]:
df_pct_change

In [ ]:
df_2_5_change

In [ ]:
df_pct_change = data1.pct_change()
df_2_5_change = data1.pct_change(3).shift(-4)
len(df_pct_change), len(df_2_5_change)

In [ ]:
df_pct_change

In [ ]:
df_2_5_change

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)

data1 = data.ix[:, :10]


df_pct_change = data1.pct_change()
df_2_5_change = data1.pct_change(3).shift(-4)

# logreturn = (data1 / data1.shift(1)).apply(np.log)
# df_vol = logreturn.rolling(window=5,center=False).std().ix[1:]

num = len(data1.columns)
xs = 5
ys = int(num/5)+1

for i, c in enumerate(data1.columns):
    x = i % xs
    y = int(i/xs)+ys
    ax = fig.add_subplot(xs, ys, i+1)
    ax.set_title(c.symbol)
    ax.scatter(df_pct_change[symbols(c)], 
               df_2_5_change[symbols(c)],
#                c=df_vol[symbols(c)],cmap=cm.Reds
               )
    